In [4]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [12]:
def movieNames():
    movie_names = {}
    with open('/ml-100k/u.item') as f:
        for line in f:
            fields = line.split('|')
            movie_names[int(fields[0])] = fileds[1]
    
    return  movie_names

In [6]:
def parseInput(line):
    fields = line.split()
    return (int (fields[1]), float (fields[2]), 1.0)

In [7]:
spark = SparkSession.builder.appName("Spark").getOrCreate()

22/01/27 11:10:51 WARN Utils: Your hostname, Ayushmans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
22/01/27 11:10:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/27 11:10:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/27 11:10:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [13]:
# getting movie names
movieNames = movieNames()

FileNotFoundError: [Errno 2] No such file or directory: '/ml-100k/u.item'